In [2]:
from dotenv import load_dotenv
load_dotenv()

import streamlit as st
import os
import google.generativeai as genai

genai.configure(api_key=os.getenv(""))

## function to load Gemini Pro model and get repsonses
model=genai.GenerativeModel("gemini-pro")
chat = model.start_chat(history=[])
def get_gemini_response(question):

    response=chat.send_message(question,stream=True)
    return response

##initialize our streamlit app

st.set_page_config(page_title="Q&A Demo")

st.header("Gemini LLM Application")

# Initialize session state for chat history if it doesn't exist
if 'chat_history' not in st.session_state:
    st.session_state['chat_history'] = []

input=st.text_input("Input: ",key="input")
submit=st.button("Ask the question")

if submit and input:
    response=get_gemini_response(input)
    # Add user query and response to session state chat history
    st.session_state['chat_history'].append(("You", input))
    st.subheader("The Response is")
    for chunk in response:
        st.write(chunk.text)
        st.session_state['chat_history'].append(("Bot", chunk.text))
st.subheader("The Chat History is")

for role, text in st.session_state['chat_history']:
    st.write(f"{role}: {text}")





2024-06-21 03:29:06.144 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-06-21 03:29:06.162 Session state does not function when running a script without `streamlit run`


In [4]:
!pip install streamlit google-generativeai pyngrok


In [5]:
%%writefile app.py
import streamlit as st
import os
from dotenv import load_dotenv
import google.generativeai as genai

load_dotenv()

genai.configure(api_key=os.getenv(""))

## function to load Gemini Pro model and get repsonses
model = genai.GenerativeModel("gemini-pro")
chat = model.start_chat(history=[])
def get_gemini_response(question):
    response = chat.send_message(question, stream=True)
    return response

##initialize our streamlit app
st.set_page_config(page_title="Q&A Demo")

st.header("Gemini LLM Application")

# Initialize session state for chat history if it doesn't exist
if 'chat_history' not in st.session_state:
    st.session_state['chat_history'] = []

input = st.text_input("Input: ", key="input")
submit = st.button("Ask the question")

if submit and input:
    response = get_gemini_response(input)
    # Add user query and response to session state chat history
    st.session_state['chat_history'].append(("You", input))
    st.subheader("The Response is")
    for chunk in response:
        st.write(chunk.text)
        st.session_state['chat_history'].append(("Bot", chunk.text))

st.subheader("The Chat History is")
for role, text in st.session_state['chat_history']:
    st.write(f"{role}: {text}")


Writing app.py


In [10]:
import os
from dotenv import load_dotenv
import gradio as gr
import google.generativeai as genai

load_dotenv()

genai.configure(api_key=os.getenv(""))

## function to load Gemini Pro model and get responses
model = genai.GenerativeModel("gemini-pro")
chat = model.start_chat(history=[])

def get_gemini_response(question):
    response = chat.send_message(question, stream=True)
    full_response = "".join([chunk.text for chunk in response])
    return full_response

# Gradio interface
def gradio_interface(input_text):
    response = get_gemini_response(input_text)
    return response

# Create a Gradio interface
interface = gr.Interface(fn=gradio_interface,
                         inputs=gr.inputs.Textbox(lines=2, placeholder="Enter your question here..."),
                         outputs="text",
                         title="Gemini LLM Application",
                         description="Ask questions and get responses from the Gemini LLM.")

# Launch the Gradio interface
interface.launch()



AttributeError: module 'gradio' has no attribute 'inputs'

In [11]:
!pip install google-generativeai translate


In [17]:
import os
import google.generativeai as genai
import re
from translate import Translator

# Set the Gemini API key (replace with your actual key)
os.environ["GOOGLE_API_KEY"] = "YOUR_GEMINI_API_KEY"
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

def extract_locations(user_input_text):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input_text)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

def generate_route_description(start_location, end_location):
    # Prompt for the Gemini API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = genai.generate_text(
        prompt,
        model="gemini-pro",
        max_output_tokens=500
    )

    return response.text.strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected

    # Initialize translator
    translator = Translator(to_lang=target_language)

    # Split route description into manageable chunks (e.g., sentences)
    sentences = route_description.split('. ')
    translated_sentences = []

    # Translate each sentence and join them back together
    for sentence in sentences:
        translated_sentence = translator.translate(sentence)
        translated_sentences.append(translated_sentence)

    translated_description = '. '.join(translated_sentences)

    return translated_description

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    # Use a different variable name to avoid overwriting the input() function
    user_input_text = input("Enter your request: ")
    if user_input_text.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input_text)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    # Ask user for desired language for route description
    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.


TypeError: 'str' object is not callable

In [18]:
import os
import google.generativeai as genai
from translate import Translator

# Set the Gemini API key (replace with your actual key)
os.environ["GOOGLE_API_KEY"] = ""
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))


In [19]:
def generate_route_description(start_location, end_location):
    # Prompt for the Gemini API
    prompt = f"Provide a detailed driving route from {start_location} to {end_location}. Include major turns, landmarks, and distances."

    response = genai.generate_text(
        prompt,
        model="gemini-pro",
        max_output_tokens=500
    )

    return response.text.strip()

def translate_description(route_description, target_language):
    if target_language == 'en':
        return route_description  # No translation needed if English is selected

    # Initialize translator
    translator = Translator(to_lang=target_language)

    # Split route description into manageable chunks (e.g., sentences)
    sentences = route_description.split('. ')
    translated_sentences = [translator.translate(sentence) for sentence in sentences]

    translated_description = '. '.join(translated_sentences)

    return translated_description


In [20]:
import re

def extract_locations(user_input_text):
    # Regular expressions to extract locations from input
    patterns = [
        re.compile(r"take me from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE),
        re.compile(r"i want to go from\s+(.+?)\s+to\s+(.+)", re.IGNORECASE)
    ]

    for pattern in patterns:
        match = pattern.search(user_input_text)
        if match:
            start_location = match.group(1).strip()
            end_location = match.group(2).strip()
            return start_location, end_location
    return None, None

# Run the chatbot in a loop
print("Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.")
while True:
    user_input_text = input("Enter your request: ")
    if user_input_text.lower() == 'exit':
        print('Exiting')
        break

    start_location, end_location = extract_locations(user_input_text)
    if not start_location or not end_location:
        print("Invalid input format. Please use phrases like 'take me from [start location] to [end location]', 'from [start location] to [end location]', or 'I want to go from [start location] to [end location]'.")
        continue

    route_description = generate_route_description(start_location, end_location)
    print("Route Description (in English):", route_description)

    # Ask user for desired language for route description
    target_language = input("Enter language code for translation (e.g., 'hi' for Hindi): ").lower()

    translated_description = translate_description(route_description, target_language)
    print(f"Route Description (in {target_language}):", translated_description)


Greetings! I am SARATHI, developed by NIT Calicut students. Type 'exit' to quit.


TypeError: 'str' object is not callable

In [21]:
!pip install -q -U google-generativeai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.1/163.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.3/717.3 kB 19.7 MB/s eta 0:00:00


In [22]:
import google.generativeai as genai
GOOGLE_API_KEY = ""
genai.configure(api_key=GOOGLE_API_KEY)


In [23]:
model = genai.GenerativeModel('gemini-pro')

In [25]:
question = 'What is the capital of Argentina?'
response = model.generate_content(question)
response.resolve()
response.text
chat = model.start_chat(history=[])
countries = ['Chile', 'Bolivia']

for country in countries:
    message = f'What is the capital of {country}?'
    chat.send_message(message)

chat.history
response = chat.send_message('From which countries did I ask you the capital city?')
response.resolve()
response.text

2024-06-21 03:49:27.798 200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 4410.82ms
2024-06-21 03:49:29.672 200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 1871.06ms
2024-06-21 03:49:32.714 200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3035.74ms
2024-06-21 03:49:35.582 200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 2859.09ms


'Chile and Bolivia'

In [26]:
import time
import gradio as gr

chat = model.start_chat(history=[])

# Transform Gradio history to Gemini format
def transform_history(history):
    new_history = []
    for chat in history:
        new_history.append({"parts": [{"text": chat[0]}], "role": "user"})
        new_history.append({"parts": [{"text": chat[1]}], "role": "model"})
    return new_history

def response(message, history):
    global chat
    # The history will be the same as in Gradio, the 'Undo' and 'Clear' buttons will work correctly.
    chat.history = transform_history(history)
    response = chat.send_message(message)
    response.resolve()

    # Each character of the answer is displayed
    for i in range(len(response.text)):
        time.sleep(0.05)
        yield response.text[: i+1]

gr.ChatInterface(response,
                 title='Gemini Chat',
                 textbox=gr.Textbox(placeholder="Question to Gemini"),
                 retry_btn=None).launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://f125722cbf60a65a2e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


2024-06-21 03:50:42.588 200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 3221.88ms
2024-06-21 03:51:40.612 200 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 5618.42ms
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/gradio/queueing.py", line 532, in process_events
    response = await route_utils.call_process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/route_utils.py", line 276, in call_process_api
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1928, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.10/dist-packages/gradio/blocks.py", line 1526, in call_function
    prediction = await utils.async_iteration(iterator)
  File "/usr/local/lib/python3.10/dist-packages/gradio/utils.py", line 656, in async_iteration
    return await iterator._

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f125722cbf60a65a2e.gradio.live


In [24]:
!pip install --upgrade gradio

In [9]:
!pip install gradio google-generativeai
!pip install --upgrade gradio

In [3]:
!streamlit run app.py

Usage: streamlit run [OPTIONS] TARGET [ARGS]...
Try 'streamlit run --help' for help.

Error: Invalid value: File does not exist: app.py


In [1]:
!pip install streamlit
!pip install google-generativeai
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.1 MB/s eta 0:00:00
